This notebook shows the basic steps that are involved in movement detection. Much of what's in here can be encaspulated by higher-order functions or classes, but if you want to have full control over things, have a look at the following.

In [1]:
import birdwatcher as bw

## Create a background subtractor object with suitable parameters

Create a background subtractor object with default parameters. This object basically encapsulates the algorithm that determines the background from a history of images, and uses that to find what is not background in the current image.

In [2]:
bgs = bw.BackgroundSubtractorMOG2()

What are the parameters?

In [3]:
bgs.get_params()

{'BackgroundRatio': 0.10000000149011612,
 'ComplexityReductionThreshold': 0.05000000074505806,
 'History': 5,
 'NMixtures': 7,
 'ShadowThreshold': 0.5,
 'ShadowValue': 127,
 'VarInit': 15.0,
 'VarMax': 75.0,
 'VarMin': 4.0,
 'VarThreshold': 10.0,
 'VarThresholdGen': 9.0,
 'class': "<class 'birdwatcher.movementdetection.BackgroundSubtractorMOG2'>"}

Let's change two of them:

In [4]:
bgs.set_params(VarThreshold=70, NMixtures=8)

In [5]:
bgs.get_params()

{'BackgroundRatio': 0.10000000149011612,
 'ComplexityReductionThreshold': 0.05000000074505806,
 'History': 5,
 'NMixtures': 8,
 'ShadowThreshold': 0.5,
 'ShadowValue': 127,
 'VarInit': 15.0,
 'VarMax': 75.0,
 'VarMin': 4.0,
 'VarThreshold': 70.0,
 'VarThresholdGen': 9.0,
 'class': "<class 'birdwatcher.movementdetection.BackgroundSubtractorMOG2'>"}

This could also have been done at intstantiation:

In [6]:
bgs = bw.BackgroundSubtractorMOG2(VarThreshold=70, NMixtures=8)

## Create a movement detector object based on this background subtractor

A movement detector object is used so detect movement, based on a supplied background subtractor. There are some convenient parameters to focus on specific parts of an image, for example.

In [7]:
movementdector = bw.MovementDetector(bgsubtractor=bgs, learningrate=-1, ignore_firstnframes=0, 
                 focus_rectcoord=None, ignore_rectcoord=None, downscale=None, morphologyex=2)

## Create a video object (see separate notebook on this)

In [8]:
vf = bw.testvideosmall()

## Create a coordinate array for storage of the results

A coordinate array really is just a Darr ragged array (see separate library). This makes it easy to read the data in other environments, e.g. R. We save some metadata so that we later know what we did.

In [9]:
coords = bw.create_coordarray('testcoords.darr', 
                              videofile=vf, 
                              metadata={'bgsparams': bgs.get_params(),
                                        'movdetectparams': movementdector.get_params()},
                              overwrite=True)

In [10]:
coords.metadata

{'bgsparams': {'BackgroundRatio': 0.10000000149011612, 'ComplexityReductionThreshold': 0.05000000074505806, 'History': 5, 'NMixtures': 8, 'ShadowThreshold': 0.5, 'ShadowValue': 127, 'VarInit': 15.0, 'VarMax': 75.0, 'VarMin': 4.0, 'VarThreshold': 70.0, 'VarThresholdGen': 9.0, 'class': "<class 'birdwatcher.movementdetection.BackgroundSubtractorMOG2'>"}, 'birdwatcher_version': '0.1.0+66.g026ec11.dirty', 'movdetectparams': {'class': "<class 'birdwatcher.movementdetection.MovementDetector'>", 'downscale': None, 'focus_rectcoord': None, 'ignore_firstnframes': 0, 'ignore_rectcoord': None, 'learningrate': -1, 'morphologyex': 2}, 'video': {'duration': 19.88, 'filename': 'zf20s_low.mp4', 'format': 0.0, 'fourcc': 'avc1', 'framecount': 497, 'framerate': 25.0, 'height': 720, 'shape': [1280, 720], 'width': 1280}}

## Now find changes in background of video based on what we have so far

We just loop over the video object's frames and feed it to the movement detector. The `apply` method will return two arrays. The first, `thresh` is an image where background is 0 and foreground (change/movement) is 1. The second, `idx`, is an aray with the positions (x,y) of the foreground pixels. The latter is nice for saving to file (coords array) because it is smaller. We use the jupyter `%%time` operator to see how long it takes to run this cell.

In [11]:
%%time
for frame in vf.iter_frames():
    thresh, idx = movementdector.apply(frame)
    coords.append(idx)

CPU times: user 30.5 s, sys: 3.46 s, total: 33.9 s
Wall time: 5.91 s


## Create a video of the results for easy inspection

In [12]:
%%time
coords.tovideo('testcoords.mp4')

CPU times: user 771 ms, sys: 327 ms, total: 1.1 s
Wall time: 3.04 s


## Take it together and look at a range of parameters

As an example we'll look at a range of history settings.

In [13]:
%%time
vf = bw.testvideosmall()
for history in (2,3,4,5):
    bgs = bw.BackgroundSubtractorMOG2(History=history, VarThreshold=50)
    movementdector = bw.MovementDetector(bgsubtractor=bgs, morphologyex=2)
    basefilename = f'testcoords_hist{history}'
    coords = bw.create_coordarray(f'{basefilename}.darr', 
                                  videofile=vf, 
                                  metadata={'bgsparams': bgs.get_params(),
                                            'bgstype': 'MOG2'},
                                  overwrite=True)
    for frame in vf.iter_frames():
        thresh, idx = movementdector.apply(frame)
        coords.append(idx)
    coords.tovideo(f'{basefilename}.mp4')

CPU times: user 1min 52s, sys: 15.7 s, total: 2min 8s
Wall time: 35.4 s
